In [1]:
import pandas as pd
import numpy as np

from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [2]:
from datetime import date, timedelta

start = '2021-06-01'
end = '2022-05-27'

start_date = date.fromisoformat(start)
end_date = date.fromisoformat(end)

date_range = [
    (end_date - timedelta(days=i)).strftime('%d-%m-%Y')
    for i
    in range((end_date - start_date).days)
]

In [ ]:
print(cg.get_coin_history_by_id(id='osmosis', date='27-05-2022'))

In [4]:
cgk_mk_data = {
    'date':[],
    'price':[],
    'mc':[],
    'circ_supply':[]
}

for date in date_range:
    cgk_mk_data['date'].append(date)

    try:
        response_data = cg.get_coin_history_by_id(id='osmosis', date=date, localization=False)['market_data']
        cgk_mk_data['price'].append(response_data['current_price']['usd'])
        cgk_mk_data['mc'].append(response_data['market_cap']['usd'])
        cgk_mk_data['circ_supply'].append(response_data['market_cap']['usd'] / response_data['current_price']['usd'])
    except:
        cgk_mk_data['price'].append(None)
        cgk_mk_data['mc'].append(None)
        cgk_mk_data['circ_supply'].append(None)

In [5]:
df = pd.DataFrame(data=cgk_mk_data)
df

,date,price,mc,circ_supply
0,27-05-2022,1.440695,5.488961e+08,3.809938e+08
1,26-05-2022,1.589280,6.030535e+08,3.794507e+08
2,25-05-2022,1.686672,6.384569e+08,3.785305e+08
3,24-05-2022,1.771739,6.690345e+08,3.776146e+08
4,23-05-2022,1.793356,6.759319e+08,3.769090e+08
...,...,...,...,...
355,06-06-2021,NaN,NaN,NaN
356,05-06-2021,NaN,NaN,NaN
357,04-06-2021,NaN,NaN,NaN
358,03-06-2021,NaN,NaN,NaN
